# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [1]:
from gensim.models import Word2Vec, KeyedVectors
w2v_path = 'models/ru.bin'
d2v_path = 'models/d2v.bin'
bm25_path = 'models/bm25.bin'

models = {}
vectors = {}

In [4]:
# если модель без тэгов
models['w2v'] = Word2Vec.load(w2v_path)

In [2]:
# если модель с POS-тэггингом
#models['w2v'] = KeyedVectors.load_word2vec_format(w2v_path, binary=False)

NameError: name 'model_path' is not defined

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [2]:
from os import listdir, remove
from os.path import isfile, join
import codecs
import json
from load_functions import *
   
def load_docs(files):
    docs = []
    for file in files:
        file = codecs.open(join('./database/', file), 'r', 'utf-8')
        docs.append(json.load(file))
    return docs

def paragraphs(docs):
    for index, doc in enumerate(docs):
        for par in doc['paragraphs']:
            yield index, par

files = load_indexing()
docs = load_docs(files)
print(len(files))

5261


In [3]:
import re
import pymorphy2

w = re.compile('[A-zА-я]+')
morph = pymorphy2.MorphAnalyzer()

def get_words(text):
    words = w.findall(text)
    words = [morph.parse(w.lower())[0].normal_form for w in words]
    return words

def count_words(text):
    words = {}
    indoc = get_words(text)
    for word in indoc:
        word = word.lower()
        if word not in words:
            words[word] = 0
        words[word] = words[word] + 1
    return words

def preprocessing(docs):
    for doc in docs:
        if 'paragraphs' in doc:
            continue
        text = doc['text']
        new_doc = []
        for line in text.split('\n'):
            words = count_words(line)
            if len(words) > 0:
                new_doc.append(words)
        i = 0
        while i < len(new_doc) - 1:
            it, next = new_doc[i], new_doc[i+1]
            if len(it) + len(next) < 20:
                it.update(next)
                del new_doc[i+1]
            else:
                i += 1
        new_doc.insert(0, count_words(doc['title']))
        doc['paragraphs'] = new_doc

def get_w2v_vector(words):
    vec = [0 for i in range(models['w2v'].vector_size)]
    size = 0
    for word, count in words.items():
        if word not in models['w2v'].wv:
            continue
        size += count
        wv = models['w2v'].wv[word]
        vec = [vec[i] + wv[i] * count for i in range(len(vec))]
    if size == 0:
        return None
    return [i / size for i in vec]
        
def get_w2v_vectors(docs):
    vectors = []
    for index, par in paragraphs(docs):        
        vec = get_w2v_vector(par)
        if vec:
            vectors.append((index, vec))
    
    return vectors

def save_w2v_base(vectors):
    cpy = []
    for vec in vectors:
        cpy.append((vec[0], [round(i, 6) for i in vec[1]]))
    file = codecs.open('w2v_base.bin', 'w', 'utf-8')
    file.write(json.dumps(cpy, indent=1))
    file.flush()
    file.close()

preprocessing(docs)
print('done preprocessing')

done preprocessing


In [17]:
vectors['w2v'] = get_w2v_vectors(docs)
print(len(vectors['w2v']))
save_w2v_base(vectors['w2v'])

21192


## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [19]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def train_doc2vec(docs):
    tagged = []
    for index, par in paragraphs(docs):        
        td = TaggedDocument(words=list(par.keys()), tags=[index])
        tagged.append(td)

    d2v_model = Doc2Vec(vector_size=100, alpha=0.025, min_count=2, dm=1)
    d2v_model.build_vocab(tagged)
    for epoch in range(50):
        d2v_model.train(tagged,
                    total_examples=d2v_model.corpus_count,
                    epochs=d2v_model.epochs)
        # decrease the learning rate
        d2v_model.alpha -= 0.0002
        # fix the learning rate, no decay
        d2v_model.min_alpha = d2v_model.alpha
    
    d2v_model.save(d2v_path)
    return d2v_model

In [20]:
models['d2v'] = train_doc2vec(docs)
#models['d2v'] = Doc2Vec.load(d2v_path)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [25]:
def get_d2v_vectors(docs):
    vectors = []
    for index, par in paragraphs(docs):        
        vec = models['d2v'].infer_vector(list(par.keys()), epochs=10)
        vec = [float(f) for f in vec]
        vectors.append((index, vec))
    return vectors

def save_d2v_base(vectors):
    cpy = []
    for vec in vectors:
        cpy.append((vec[0], [round(i, 6) for i in vec[1]]))
    file = codecs.open('d2v_base.bin', 'w', 'utf-8')
    file.write(json.dumps(cpy, indent=1))
    file.flush()
    file.close()

In [26]:
vectors['d2v'] = get_d2v_vectors(docs)
print(len(vectors['d2v']))
save_d2v_base(vectors['d2v'])

23055


# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

In [21]:


import importlib
import bm25.parse
import bm25.query
importlib.reload(bm25.parse)
importlib.reload(bm25.query)

from bm25.query import QueryProcessor
from bm25.parse import Corpus

def build_inv_index(docs):
    corp = Corpus()
    for index, par in paragraphs(docs):
        corp.add(index, list(par.keys()))
    
    proc = QueryProcessor(corp)
    proc.save(bm25_path)
    return proc

models['bm25'] = build_inv_index(docs)
#models['bm25'] = QueryProcessor.load(bm25_path)
print('loaded')

loaded


Функция измерения близости между векторами нам пригодится:

In [43]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [1]:
def search_w2v(document):    
    vec = get_w2v_vector(count_words(document))
    results = [(i, similarity(w2v_vec, vec)) for i, w2v_vec in enumerate(vectors['w2v'])]
    results.sort(key = lambda x: x[1], reversed=True)
    return results

def search_d2v(document):
    vec = models['d2v'].infer_vector(get_words(document), epochs=10)
    vec = [float(f) for f in vec]
    results = [(i, similarity(d2v_vec, vec)) for i, d2v_vec in enumerate(vectors['d2v'])]
    results.sort(key = lambda x: x[1], reversed=True)
    return results

def search_inv_index(document):
    results = models['bm25'].run_query(get_words(document))
    return results

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [ ]:
def search(text, search_method):
    if search_method == 'inverted_index':
        search_result = search_inv_index(text)
    elif search_method == 'word2vec':
        search_result = search_w2v(text)
    elif search_method == 'doc2vec':
        search_result = search_d2v(text)
    else:
        raise TypeError('unsupported search method')
    print(search_result)